In [1]:
import itertools
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils 
from torch.autograd import Variable

from torchvision import transforms

import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
df_label = pd.read_csv('./Platin_Data/Semi_Final/common_patient.csv', sep = '\t', index_col = 0)

In [3]:
df_label

,PLATINUM_STATUS
PATIENT_ID,
TCGA-04-1331,Sensitive
TCGA-04-1332,Sensitive
TCGA-04-1347,Sensitive
TCGA-04-1362,Resistant
TCGA-04-1364,Resistant
...,...
TCGA-61-2098,Sensitive
TCGA-61-2109,Sensitive
TCGA-61-2110,Resistant


In [4]:
df_label.loc[df_label['PLATINUM_STATUS'] == 'Sensitive', 'label'] = 0
df_label.loc[df_label['PLATINUM_STATUS'] == 'Resistant', 'label'] = 1

In [5]:
df_label

,PLATINUM_STATUS,label
PATIENT_ID,,
TCGA-04-1331,Sensitive,0.0
TCGA-04-1332,Sensitive,0.0
TCGA-04-1347,Sensitive,0.0
TCGA-04-1362,Resistant,1.0
TCGA-04-1364,Resistant,1.0
...,...,...
TCGA-61-2098,Sensitive,0.0
TCGA-61-2109,Sensitive,0.0
TCGA-61-2110,Resistant,1.0


In [12]:
df_data = pd.read_csv('./Platin_Data/Semi_Final/PC_tpm_200_minmax.csv', sep = '\t', index_col = 0)

In [13]:
df_data

,TCGA-04-1331,TCGA-04-1332,TCGA-04-1347,TCGA-04-1362,TCGA-04-1364,TCGA-04-1365,TCGA-04-1514,TCGA-04-1530,TCGA-04-1536,TCGA-04-1542,...,TCGA-61-2000,TCGA-61-2008,TCGA-61-2009,TCGA-61-2092,TCGA-61-2097,TCGA-61-2098,TCGA-61-2109,TCGA-61-2110,TCGA-61-2111,TCGA-61-2113
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000131096,0.001124,0.000566,0.000096,0.002925,0.001883,0.000146,0.000412,0.200155,0.000272,0.000401,...,0.000119,0.000052,0.000138,0.000038,0.000651,0.000309,0.001277,0.000163,0.023144,0.000137
ENSG00000187581,0.074279,0.001385,0.002826,0.016522,1.000000,0.015051,0.017779,0.000000,0.001618,0.018865,...,0.015682,0.001707,0.222856,0.000508,0.005218,0.154442,0.112554,0.006289,0.096016,0.010557
ENSG00000047936,0.149561,0.002034,0.000060,0.002256,0.000016,0.007940,0.000027,0.000705,0.000014,0.000171,...,0.000045,0.000044,0.000303,0.000000,0.000134,0.000215,0.005764,0.000069,0.000012,0.000103
ENSG00000186198,0.020867,0.004467,0.004142,0.005812,0.020709,0.005042,0.001216,0.001000,0.000285,0.001728,...,0.005363,0.001585,0.002821,0.000372,0.003824,0.001345,0.027494,0.000263,0.001709,0.002652
ENSG00000179914,0.008411,0.056473,0.001344,0.001347,0.000372,0.001071,0.001127,0.000742,0.004618,0.000000,...,0.032674,0.001082,0.001927,0.000023,0.006381,0.001390,0.258720,0.030643,0.001957,0.000410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000153822,0.202502,0.002711,0.004158,0.001322,0.000091,0.014324,0.000079,0.004260,0.003334,0.000000,...,0.000672,0.000008,0.000420,0.006529,0.000149,0.000120,0.013265,0.014825,0.000069,0.001336
ENSG00000197616,0.009398,0.000928,0.000315,0.022576,0.002524,0.000320,0.026711,0.000975,0.000014,0.007845,...,0.000700,0.000025,0.000859,0.000000,0.012233,0.010409,0.000838,0.000120,0.002083,0.000067
ENSG00000112499,0.000136,0.000092,0.000016,0.000036,0.000020,0.000008,0.000070,0.000000,0.000004,0.000000,...,0.000058,0.000006,0.000011,0.000011,0.000029,0.000003,0.000723,0.000010,0.000008,0.000017


In [14]:
df_data_mod = df_data.T

In [15]:
df_data_mod

ID,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,ENSG00000136944,ENSG00000182870,...,ENSG00000048545,ENSG00000188076,ENSG00000048462,ENSG00000187094,ENSG00000106038,ENSG00000153822,ENSG00000197616,ENSG00000112499,ENSG00000111249,ENSG00000203909
TCGA-04-1331,0.001124,0.074279,0.149561,0.020867,0.008411,0.372549,0.000000,0.074310,0.063072,0.002291,...,0.001769,1.000000,0.012713,0.018254,0.000933,0.202502,0.009398,0.000136,0.000482,0.005991
TCGA-04-1332,0.000566,0.001385,0.002034,0.004467,0.056473,0.046285,0.000000,0.005921,0.029110,0.000781,...,0.000264,0.002173,0.000000,0.000000,0.000209,0.002711,0.000928,0.000092,0.000072,0.000000
TCGA-04-1347,0.000096,0.002826,0.000060,0.004142,0.001344,0.793124,0.000461,0.001680,0.000038,0.000037,...,0.000000,0.132969,0.002902,0.000000,0.000000,0.004158,0.000315,0.000016,0.000010,0.224272
TCGA-04-1362,0.002925,0.016522,0.002256,0.005812,0.001347,0.015423,0.003234,0.054898,0.010333,0.001136,...,0.000787,0.000000,0.000000,0.019489,0.000996,0.001322,0.022576,0.000036,0.000539,0.000000
TCGA-04-1364,0.001883,1.000000,0.000016,0.020709,0.000372,0.245882,0.000000,0.010063,0.000123,0.000088,...,0.011812,0.000000,0.001123,0.001152,0.000275,0.000091,0.002524,0.000020,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-61-2098,0.000309,0.154442,0.000215,0.001345,0.001390,0.016812,0.000855,0.233778,0.002315,0.000060,...,0.000050,0.001234,0.000108,0.000442,0.000040,0.000120,0.010409,0.000003,0.001646,0.001015
TCGA-61-2109,0.001277,0.112554,0.005764,0.027494,0.258720,1.000000,0.003060,0.063795,0.004804,0.005207,...,0.000447,0.000000,0.053940,0.001317,0.000000,0.013265,0.000838,0.000723,0.000452,0.000000
TCGA-61-2110,0.000163,0.006289,0.000069,0.000263,0.030643,0.001324,0.000000,0.015528,0.001865,0.008710,...,0.000000,0.000705,0.001476,0.002523,0.000000,0.014825,0.000120,0.000010,0.000007,0.002609
TCGA-61-2111,0.023144,0.096016,0.000012,0.001709,0.001957,0.002696,0.021030,0.000892,0.000157,0.000115,...,0.000033,0.000538,0.003381,0.000000,0.000000,0.000069,0.002083,0.000008,0.000094,0.000000


In [16]:
#df_label_mi = pd.read_csv('./Platin_Data/miRNA_patient.csv', sep = '\t', index_col = 0)

In [17]:
df_data_mi = pd.read_csv('./Platin_Data/Semi_Final/MIR_rpm_100_minmax.csv', sep = '\t', index_col = 0)

In [18]:
df_data_mi = df_data_mi.T

In [19]:
df_data_lnc = pd.read_csv('./Platin_Data/Semi_Final/LNC_tpm_100_minmax.csv', sep = '\t', index_col = 0)

In [20]:
df_data_lnc

,TCGA-04-1331,TCGA-04-1332,TCGA-04-1347,TCGA-04-1362,TCGA-04-1364,TCGA-04-1365,TCGA-04-1514,TCGA-04-1530,TCGA-04-1536,TCGA-04-1542,...,TCGA-61-2000,TCGA-61-2008,TCGA-61-2009,TCGA-61-2092,TCGA-61-2097,TCGA-61-2098,TCGA-61-2109,TCGA-61-2110,TCGA-61-2111,TCGA-61-2113
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000233048,0.000801,0.000000,0.000000,0.000000,0.003353,0.000284,0.000174,0.000000,0.000000,0.000297,...,0.000000,0.000510,0.000000,0.000000,0.000000,0.000316,0.000031,0.000000,0.000452,0.000000
ENSG00000083622,0.000000,0.001047,0.000000,0.001600,0.000000,0.000581,0.000000,0.003550,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000038,0.000021,0.003009,0.007278
ENSG00000269994,0.000294,0.000547,0.001422,0.002408,0.000000,0.001458,0.000447,0.003711,0.002415,0.000000,...,0.003453,0.001311,0.000431,0.000268,0.000000,0.000541,0.000047,0.000036,0.029038,0.004058
ENSG00000258752,0.098097,0.002003,0.002169,0.003266,0.050686,0.005930,0.248185,0.013590,0.004913,0.000000,...,0.000000,0.053335,0.002633,0.040338,0.001441,0.029168,0.000032,0.000164,0.001181,0.008254
ENSG00000249790,0.001326,0.002467,0.006410,0.001810,0.002100,0.003286,0.010075,0.000000,0.000000,0.000000,...,0.010379,0.011822,0.011671,0.006041,0.014375,0.006709,0.000071,0.000000,0.000000,0.018295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000256904,0.011139,0.024870,0.000000,0.015204,0.000000,0.000000,0.000000,0.000000,0.045749,0.000000,...,0.000000,0.000000,0.016342,0.000000,0.000000,0.000000,0.000000,0.000340,0.000000,0.000000
ENSG00000232197,0.000000,0.001466,1.000000,0.005378,1.000000,0.151355,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.028724,0.000000,0.038062,0.010317,0.000000,0.023344,0.000000
ENSG00000224957,0.000161,0.000060,0.001206,0.000000,0.000013,0.000000,0.000049,0.000244,0.000000,0.000000,...,0.000063,0.000143,0.000094,0.000147,0.000000,0.000030,0.000002,0.000008,0.000032,0.000222


In [21]:
df_data_lnc = df_data_lnc.T

In [22]:
#df_mi = pd.concat([df_label_mi,df_data_mi], axis = 1)

In [23]:
df = pd.concat([df_label,df_data_mod,df_data_mi,df_data_lnc], axis = 1)

In [24]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size = 0.2)

In [25]:
train

,PLATINUM_STATUS,label,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,...,ENSG00000272457,ENSG00000260426,ENSG00000250685,ENSG00000280916,ENSG00000255580,ENSG00000256904,ENSG00000232197,ENSG00000224957,ENSG00000259181,ENSG00000229520
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-09-0366,Resistant,1.0,0.000504,0.018500,0.000589,0.002169,0.021954,0.019088,0.000000,0.008939,...,0.000000,0.000000,0.002989,0.046552,0.000235,0.000000,0.117877,0.000141,0.000000,0.008536
TCGA-23-1026,Sensitive,0.0,0.000102,0.002573,0.000110,0.001225,0.000437,0.007742,0.010910,0.000072,...,0.000000,0.008825,0.000913,0.002880,0.000000,0.000000,0.003236,0.000000,0.000000,0.000000
TCGA-61-2097,Sensitive,0.0,0.000651,0.005218,0.000134,0.003824,0.006381,0.026279,0.000000,0.008049,...,0.048242,0.679474,0.000000,0.025340,0.000000,0.000000,0.000000,0.000000,0.000000,0.045882
TCGA-09-0364,Sensitive,0.0,0.004684,0.008449,0.000010,0.001061,0.232067,0.297989,0.015355,0.336318,...,0.037940,0.000000,0.002106,0.192644,0.000000,0.105531,0.000000,0.003963,0.002966,0.000000
TCGA-09-2053,Sensitive,0.0,0.004013,0.002056,0.001737,0.003465,0.051682,0.276771,0.000335,0.092957,...,0.011494,0.000000,0.001276,0.000000,0.000802,0.000000,0.061065,0.000942,0.001797,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-61-1918,Sensitive,0.0,0.000226,0.003562,0.000407,0.001879,0.001355,0.027825,0.000000,0.009611,...,0.000000,0.000000,0.000000,0.000543,0.000216,0.000000,0.000000,0.000000,0.000000,0.000000
TCGA-24-1474,Sensitive,0.0,0.000031,0.002280,0.000019,0.000535,0.000310,0.003696,0.000149,0.000542,...,0.111469,0.000000,0.018562,0.126860,0.000000,0.031005,0.021933,0.000627,0.002905,0.141355
TCGA-04-1536,Sensitive,0.0,0.000272,0.001618,0.000014,0.000285,0.004618,0.001988,0.000106,0.000373,...,0.000000,0.044126,0.000000,0.000000,0.000000,0.045749,0.000000,0.000000,0.000000,0.000000


In [26]:
train.to_csv("train_platin_total_lnc__100_400.csv", mode='w')

In [27]:
val

,PLATINUM_STATUS,label,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000186198,ENSG00000179914,ENSG00000186897,ENSG00000138136,ENSG00000139219,...,ENSG00000272457,ENSG00000260426,ENSG00000250685,ENSG00000280916,ENSG00000255580,ENSG00000256904,ENSG00000232197,ENSG00000224957,ENSG00000259181,ENSG00000229520
PATIENT_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-61-1725,Sensitive,0.0,0.000417,0.008346,0.000000,0.004893,0.009526,0.012712,0.000000,0.001737,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000024,0.000771,0.000000
TCGA-25-1315,Resistant,1.0,0.000710,0.036490,0.000071,0.011308,0.008925,0.049174,0.000340,0.028984,...,0.003411,0.000458,0.000189,0.006420,0.000000,0.001897,0.001007,0.000008,0.000089,0.000000
TCGA-13-1487,Sensitive,0.0,0.000000,0.004556,0.005094,0.002337,0.001238,0.781805,0.000000,0.000254,...,0.000000,0.000000,0.000000,0.005778,0.000000,0.000000,0.000000,0.000053,0.003440,0.000000
TCGA-61-2111,Sensitive,0.0,0.023144,0.096016,0.000012,0.001709,0.001957,0.002696,0.021030,0.000892,...,0.059319,0.000000,0.000000,0.000000,0.000000,0.000000,0.023344,0.000032,0.003091,0.162982
TCGA-25-1318,Resistant,1.0,0.000055,0.000000,0.000352,0.007813,0.000988,0.204179,0.000790,0.002071,...,0.000000,0.009608,0.000000,0.001568,0.000625,0.000000,0.000000,0.000058,0.000000,0.000000
TCGA-23-1023,Sensitive,0.0,0.000022,0.015233,0.002357,0.001008,0.002871,0.001690,0.000160,0.001808,...,0.003508,0.000000,0.000000,0.004913,0.000122,0.000000,0.000000,0.000085,0.000183,0.000000
TCGA-24-1557,Resistant,1.0,0.000367,0.005775,0.000306,0.002201,0.009573,0.006809,0.000000,0.012577,...,0.000000,0.000000,0.000000,0.003749,0.000000,0.000000,0.000000,0.000069,0.002232,0.000000
TCGA-04-1347,Sensitive,0.0,0.000096,0.002826,0.000060,0.004142,0.001344,0.793124,0.000461,0.001680,...,0.290409,0.000000,0.005373,0.000000,0.001689,0.000000,1.000000,0.001206,0.001261,0.000000
TCGA-13-0884,Sensitive,0.0,0.002014,0.000000,0.004485,0.038014,0.000000,0.422523,0.000000,0.001522,...,0.000000,0.000000,0.000000,0.000118,0.000000,0.000249,0.000264,0.000004,0.000000,0.000284


In [28]:
val.to_csv("val_platin_total_lnc__100_400.csv", mode='w')

In [194]:
train = pd.read_csv("train_platin_total_lnc__100_400.csv")
val = pd.read_csv("val_platin_total_lnc__100_400.csv")

In [195]:
trn_X_pd = train.drop(["PATIENT_ID","PLATINUM_STATUS","label"],axis=1).values
trn_y_pd = train.label.values

print("Information of Training Data")
print("Samples : {}".format(trn_X_pd.shape[0]))
print("Features : {}".format(trn_X_pd.shape[1]))

Information of Training Data
Samples : 168
Features : 400


In [196]:
val_X_pd = val.drop(["PATIENT_ID","PLATINUM_STATUS","label"],axis=1).values
val_y_pd = val.label.values

print("Information of Training Data")
print("Samples : {}".format(val_X_pd.shape[0]))
print("Features : {}".format(val_X_pd.shape[1]))

Information of Training Data
Samples : 42
Features : 400


In [197]:
item = []
item_2 = []

print( (val_y_pd == 1.0).sum())
print( (val_y_pd == 0.0).sum())

print( (trn_y_pd == 1.0).sum())
print( (trn_y_pd == 0.0).sum())

14
28
47
121


In [198]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5 # Number of K-fold Splits

splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(trn_X_pd, trn_y_pd))
splits[:5]

[(array([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  17,  18,  21,  22,  23,  24,  25,  26,  29,  31,  32,
          33,  34,  35,  36,  37,  38,  39,  40,  41,  44,  45,  46,  47,
          48,  49,  50,  51,  52,  53,  55,  56,  57,  58,  59,  60,  61,
          62,  63,  65,  66,  67,  68,  69,  70,  72,  73,  74,  76,  77,
          81,  83,  84,  85,  86,  87,  88,  89,  90,  92,  93,  94,  96,
          97,  98,  99, 100, 105, 106, 107, 108, 109, 110, 112, 114, 115,
         116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 128, 129,
         130, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145,
         146, 148, 149, 150, 151, 154, 155, 156, 157, 159, 160, 163, 164,
         165, 166, 167]),
  array([  1,  16,  19,  20,  27,  28,  30,  42,  43,  54,  64,  71,  75,
          78,  79,  80,  82,  91,  95, 101, 102, 103, 104, 111, 113, 127,
         131, 133, 143, 147, 152, 153, 158, 161, 162])),
 (array([  1,   2,   3,   4, 

In [199]:
for i, (train_idx, valid_idx) in enumerate(splits):

    print( (trn_y_pd[valid_idx.astype(int)] == 1.0).sum())
    print( (trn_y_pd[valid_idx.astype(int)] == 0.0).sum())
    
    #print( (trn_y_pd[train_idx.astype(int)] == 1.0).sum())
    #print( (trn_y_pd[train_idx.astype(int)] == 0.0).sum())

10
25
10
24
9
24
9
24
9
24


In [200]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)


In [201]:
batch_size = 5

In [202]:
class DNN_seq_1(nn.Module):
    def __init__(self):
        super(DNN_seq_1, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
        )
        self.classifier2 = nn.Sequential(

            torch.nn.Linear(100, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250,1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [203]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq(nn.Module):
    def __init__(self):
        super(DNN_seq, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 200, bias=True),
            torch.nn.BatchNorm1d(200),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(200, 300, bias=True),
            torch.nn.BatchNorm1d(300),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(300, 300, bias=True),
            torch.nn.BatchNorm1d(300),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            torch.nn.Linear(300, 300, bias=True),
            torch.nn.BatchNorm1d(300),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(300, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [204]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq_2(nn.Module):
    def __init__(self):
        super(DNN_seq_2, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(150, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(100, 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            torch.nn.Linear(150, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.5),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            torch.nn.BatchNorm1d(1)
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [205]:
model = DNN_seq_2()

In [206]:
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [207]:
from torch.optim.optimizer import Optimizer

import time # ??

from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


In [208]:
print(trn_X_pd.shape[0])
          
train_preds = np.zeros(trn_X_pd.shape[0])

## Addiction
train_y_sort = np.zeros(trn_X_pd.shape[0])

test_preds = np.zeros(val_X_pd.shape[0])


train_target = train.label

168


In [52]:
avg_losses_f = []
avg_val_losses_f = []

for i, (train_idx, valid_idx) in enumerate(splits):
    
    ## ???
    x_train_fold = torch.tensor(trn_X_pd[train_idx.astype(int)], dtype=torch.float)   # use_cuse?
    y_train_fold = torch.tensor(trn_y_pd[train_idx.astype(int), np.newaxis], dtype=torch.float)    

    x_val_fold = torch.tensor(trn_X_pd[valid_idx.astype(int)], dtype=torch.float)
    y_val_fold = torch.tensor(trn_y_pd[valid_idx.astype(int), np.newaxis], dtype=torch.float)  
    
################################################################################    
    trn_X = torch.from_numpy(trn_X_pd[train_idx.astype(int)].astype(float))
    trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
    
    # Train
    trn = Dataset(trn_X, trn_y)
    trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)  # True or False
    trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False, drop_last = False)  # True or False
    
    # Valid
    valid_X = torch.from_numpy(trn_X_pd[valid_idx.astype(int)].astype(float))
    valid_y = torch.from_numpy(trn_y_pd[valid_idx.astype(int)].astype(float))
    
    valid = Dataset(valid_X, valid_y)
    valid_loader = data_utils.DataLoader(valid, batch_size=batch_size, shuffle=False, drop_last = False) # True or False
    
    # Test
    val_X = torch.from_numpy(val_X_pd.astype(float))
    val_y = torch.from_numpy(val_y_pd.astype(float))
    val = Dataset(val_X, val_y)
    test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)
   ################################################################################    
    print(i)
    
    ## Loss function
    criterion = nn.BCELoss()
    learning_rate = 1e-3
    
    ## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
    
    model = DNN_seq_1()
    use_cuda = torch.cuda.is_available()
    num_epochs = 50
    
    if use_cuda:
        model = model.cuda()
    
    ##############################################################################
    step_size = 2000
    base_lr, max_lr = 0.001, 0.01  
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
    scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    
    
    ##############################################################################
    
    print(f'Fold {i+1}')
    
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.
        #correct = 0.   # Accuracy
        
        for batch_idx, trn in enumerate(trn_loader):
            trn_X, trn_y = trn['X'], trn['y']
            if use_cuda:
                trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
            trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
            optimizer.zero_grad()
            trn_pred = model(trn_X)

            
            if scheduler:
                #print('cycle_LR')
                scheduler.batch_step()
            #print(trn_pred.squeeze())
            #print(trn_y)
            trn_loss = criterion(trn_pred.squeeze(), trn_y)
            trn_loss.backward()
            optimizer.step()
        
            avg_loss += trn_loss.item()/len(trn_loader)
        
            #predicted = torch.max(trn_pred.data, 1)[1] 
            #correct += (predicted == trn_y).sum()
        model.eval()
        
        valid_preds_fold = np.zeros((valid_X.size(0)))
        test_preds_fold = np.zeros(val_X_pd.shape[0])  # Test
        
        avg_val_loss = 0.
        
        with torch.no_grad():
            for batch_idx, val in enumerate(valid_loader):
                val_X, val_y = val['X'], val['y']
                if use_cuda:
                    val_X, val_y = val_X.cuda(), val_y.cuda()
                val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
                optimizer.zero_grad()
                val_pred = model(val_X).detach()

            
                val_loss = criterion(val_pred.squeeze(), val_y)
        
                avg_val_loss += val_loss.item()/len(valid_loader)
            
            
                #val_pred = torch.max(val_pred, 1)[1]
                val_pred = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
                #print(val_pred.cpu().numpy())
                #print(val_pred.cpu().numpy()[:,0])
            
                valid_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (val_pred.cpu().numpy())    # modified [:,0]
            
            #valid_preds_fold_2 = 
                # Loss function chage -> plus Sigmoid
            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(epoch + 1, num_epochs, avg_loss, avg_val_loss, elapsed_time))            

# Test        
    avg_losses_f.append(avg_loss)
    avg_val_losses_f.append(avg_val_loss) 
    
    for batch_idx, test in enumerate(test_loader):
        test_X, test_y = test['X'], test['y']
        if use_cuda:
            test_X, test_y = test_X.cuda(), test_y.cuda()
        test_X, test_y = Variable(test_X).float(), Variable(test_y).float()        
        test_pred = model(test_X).detach()
        
        
        test_pred = (test_pred > 0.5).flatten().type(torch.ByteTensor)
        
        #print(test_pred)
        #print(test_pred.cpu().numpy())
        test_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (test_pred.cpu().numpy())   # modified [:,0]
        
        
    train_preds[valid_idx.astype(int)] = valid_preds_fold
    print(valid_preds_fold)
    print(trn_y_pd[valid_idx.astype(int)])
    train_y_sort[valid_idx.astype(int)] = trn_y_pd[valid_idx.astype(int)]
    test_preds += test_preds_fold / len(splits)

#predict_ = pd.DataFrame(predict)

#predict_.iloc[:,0]

#label = pd.DataFrame(val_y_pd)

#test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)    
    
print(trn_y_pd)
print(train_preds)

auc  =  round(roc_auc_score(train_y_sort,train_preds.astype(int)),4)


cnf = confusion_matrix(train_y_sort, train_preds, labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

print('AUC: ', auc)


#fpr, trp, _  =  roc_auc_score(train_y_sort,train_preds.astype(int))

#fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

#print(type(fpr))
#print(fpr)

#auc = auc(fpr, trp)
#print('AUC: ', auc(fpr, trp))
print('\n')
print('All \t loss={:.4f} \t val_loss={:.4f} \t auc={:.4f}'.format(np.average(avg_losses_f),np.average(avg_val_losses_f),auc))        
        

0
Fold 1
Epoch 1/50 	 loss=0.6161 	 val_loss=0.6521 	 time=0.13s
Epoch 2/50 	 loss=0.5052 	 val_loss=0.5938 	 time=0.11s
Epoch 3/50 	 loss=0.4408 	 val_loss=0.5745 	 time=0.11s
Epoch 4/50 	 loss=0.4180 	 val_loss=0.5723 	 time=0.11s
Epoch 5/50 	 loss=0.3157 	 val_loss=0.5825 	 time=0.11s
Epoch 6/50 	 loss=0.2713 	 val_loss=0.6199 	 time=0.12s
Epoch 7/50 	 loss=0.1842 	 val_loss=0.7045 	 time=0.10s
Epoch 8/50 	 loss=0.2965 	 val_loss=0.6633 	 time=0.11s
Epoch 9/50 	 loss=0.2291 	 val_loss=0.5707 	 time=0.11s
Epoch 10/50 	 loss=0.2481 	 val_loss=0.5822 	 time=0.11s
Epoch 11/50 	 loss=0.1580 	 val_loss=0.6458 	 time=0.15s
Epoch 12/50 	 loss=0.1930 	 val_loss=0.6818 	 time=0.18s
Epoch 13/50 	 loss=0.2565 	 val_loss=0.6330 	 time=0.13s
Epoch 14/50 	 loss=0.1943 	 val_loss=0.6887 	 time=0.15s
Epoch 15/50 	 loss=0.2131 	 val_loss=0.8184 	 time=0.12s
Epoch 16/50 	 loss=0.2359 	 val_loss=0.6598 	 time=0.11s
Epoch 17/50 	 loss=0.3043 	 val_loss=0.5393 	 time=0.11s
Epoch 18/50 	 loss=0.1918 	 val

Epoch 38/50 	 loss=0.1338 	 val_loss=0.6496 	 time=0.11s
Epoch 39/50 	 loss=0.1417 	 val_loss=0.7268 	 time=0.11s
Epoch 40/50 	 loss=0.1481 	 val_loss=0.7587 	 time=0.12s
Epoch 41/50 	 loss=0.2307 	 val_loss=0.7274 	 time=0.11s
Epoch 42/50 	 loss=0.1168 	 val_loss=0.5932 	 time=0.12s
Epoch 43/50 	 loss=0.1626 	 val_loss=0.9284 	 time=0.11s
Epoch 44/50 	 loss=0.2412 	 val_loss=0.9288 	 time=0.13s
Epoch 45/50 	 loss=0.2493 	 val_loss=1.1492 	 time=0.11s
Epoch 46/50 	 loss=0.2299 	 val_loss=0.8384 	 time=0.11s
Epoch 47/50 	 loss=0.1873 	 val_loss=0.6861 	 time=0.11s
Epoch 48/50 	 loss=0.2444 	 val_loss=0.7774 	 time=0.11s
Epoch 49/50 	 loss=0.3534 	 val_loss=0.6944 	 time=0.11s
Epoch 50/50 	 loss=0.1855 	 val_loss=0.5458 	 time=0.11s
[0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 0.]
3
Fold 4
Epoch 1/50 	 loss=0.6251 	 val_loss=0.5926 	 t

# Valid

In [53]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(valid_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(valid_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(valid_loader)*batch_size)))

tensor([0, 1, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([0, 0, 0, 1, 0], dtype=torch.uint8)
tensor([0., 0., 0., 0., 0.])
tensor([0, 0, 0, 0, 0], dtype=torch.uint8)
tensor([1., 1., 0., 1., 0.])
tensor([0, 0, 1, 1, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([1, 1, 0, 0, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 1, 1, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([0, 0, 1], dtype=torch.uint8)
tensor([0., 1., 0.])
******************** Test ********************
Loss: 1.305253028869629, Accuracy: 0.5428571701049805 %
**********************************************
Val accuracy:0.543


In [54]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(trn_y_pd[valid_idx.astype(int)], predict_.iloc[:,0])

,0
0,
2.570196e-03,0.0
6.824775e-01,0.0
6.884063e-01,1.0
4.023325e-04,0.0
4.617815e-01,0.0
2.172085e-06,0.0
3.782858e-03,0.0
1.640790e-01,0.0
8.388609e-01,0.0


In [55]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd[valid_idx.astype(int)])

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

#fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])
auc  =  round(roc_auc_score(test_p['label'],test_p['predicted_prob']),4)
#print(fpr)
#print(trp)
#print(fpr)
print('AUC: ', auc)

[[ 4  5]
 [ 9 15]]
Accuracy :  0.5757575757575758
Sensitivity :  0.4444444444444444
Specificity :  0.625
AUC:  0.5347


In [56]:
len(x_train_fold)

135

In [57]:
model = DNN_seq_1()

# Model Load

In [177]:
#model = TheModelClass(*args, **kwargs)                                                                                                     
#optimizer = TheOptimizerClass(*args, **kwargs)                                                                                             
checkpoint = torch.load("./platin_model_save/platin_model_300_100_100_model_1_150_250.pth")                                                                                  
model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
#num_epochs = checkpoint['epoch']                                                                                                           
loss = checkpoint['loss']                                                                                                                   
model.eval()  

DNN_seq_1(
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=150, bias=True)
    (1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=150, out_features=250, bias=True)
    (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=250, out_features=1, bias=True)
  )
)

# Test

In [58]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0., 1., 0., 0., 1.])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1., 0., 1., 0., 0.])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0., 1., 0., 1., 0.])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 1], dtype=torch.uint8)
tensor([1., 1.])
******************** Test ********************
Loss: 0.6741578578948975, Accuracy: 0.31111112236976624 %
**********************************************
Val accuracy:0.311


In [59]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.510388,0.0
0.510427,1.0
0.512858,0.0
0.508425,0.0
0.509437,1.0
0.508653,0.0
0.508062,1.0
0.507809,0.0
0.508631,0.0


In [60]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[14  0]
 [28  0]]
Accuracy :  0.3333333333333333
Sensitivity :  1.0
Specificity :  0.0
<class 'numpy.ndarray'>
AUC:  0.5


In [181]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_300_100_100_model_1_150_250___.pth")     

# Train Test (Except Valid)

In [61]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(trn_test_loader)*batch_size)))

tensor([[0.5117],
        [0.5083],
        [0.5079],
        [0.5089],
        [0.5081]])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([[0.5116],
        [0.5098],
        [0.5086],
        [0.5098],
        [0.5111]])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([[0.5095],
        [0.5064],
        [0.5109],
        [0.5088],
        [0.5119]])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([[0.5091],
        [0.5099],
        [0.5049],
        [0.5116],
        [0.5077]])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([[0.5070],
        [0.5076],
        [0.5074],
        [0.5064],
        [0.5072]])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([[0.5065],
        [0.5058],
        [0.5091],
        [0.5095],
        [0.5116]])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([[0.5107],
        [0.5087],
        [0.5113],
        [0.5109],
        [0.5093]])
tensor([1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([[0.5071],
        [0.5079],
        [0.5109],
        

In [62]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(trn_y_pd[train_idx.astype(int)], predict_.iloc[:,0])

,0
0,
0.511696,1.0
0.508304,0.0
0.507908,0.0
0.508913,0.0
0.508103,0.0
...,...
0.508933,1.0
0.507883,0.0
0.507486,0.0


In [63]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd[train_idx.astype(int)])

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[38  0]
 [97  0]]
Accuracy :  0.2814814814814815
Sensitivity :  1.0
Specificity :  0.0
AUC:  0.5


# .........................................................................

# Real Train stage

# .........................................................................

# Train (Total Data)

In [209]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [210]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [211]:
## Loss function
criterion = nn.BCELoss()
learning_rate = 1e-3
    
## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-7)
    
model = DNN_seq_1()
use_cuda = torch.cuda.is_available()
num_epochs = 90
    
if use_cuda:
    model = model.cuda()
    
##############################################################################
step_size = 2000
base_lr, max_lr = 0.001, 0.01  
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    

In [212]:


predict = []

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    #correct = 0.   # Accuracy
    correct_val = 0.
        
    for batch_idx, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model(trn_X)
        for i in trn_pred:
            predict.append(i.detach().numpy())
            
        if scheduler:
            #print('cycle_LR')
            scheduler.batch_step()
        #print(trn_pred.squeeze())
        #print(trn_y)
        trn_loss = criterion(trn_pred.squeeze(), trn_y)
        trn_loss.backward()
        optimizer.step()
        
        avg_loss += trn_loss.item()/len(trn_loader)
        elapsed_time = time.time() - start_time
        predicted = (trn_pred >= 0.5).flatten().type(torch.ByteTensor)
        correct_val += (predicted == trn_y).sum()
    print('*'*20, 'Train', '*'*20)
    print('Loss: {}, Accuracy: {} % \t time={:.2f}s'.format(trn_loss.item(), correct_val/(len(trn_loader)*batch_size), elapsed_time))
print('*'*46)
print("Trn accuracy:{:.3f} ".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.6415067315101624, Accuracy: 0.6424242258071899 % 	 time=0.17s
******************** Train ********************
Loss: 0.7745437622070312, Accuracy: 0.7454545497894287 % 	 time=0.13s
******************** Train ********************
Loss: 0.5653578639030457, Accuracy: 0.8242424130439758 % 	 time=0.15s
******************** Train ********************
Loss: 0.37516969442367554, Accuracy: 0.8484848737716675 % 	 time=0.13s
******************** Train ********************
Loss: 0.28370967507362366, Accuracy: 0.8787878751754761 % 	 time=0.14s
******************** Train ********************
Loss: 0.12547652423381805, Accuracy: 0.8848484754562378 % 	 time=0.14s
******************** Train ********************
Loss: 0.15186096727848053, Accuracy: 0.8909090757369995 % 	 time=0.15s
******************** Train ********************
Loss: 1.6121881008148193, Accuracy: 0.8969696760177612 % 	 time=0.13s
******************** Train ********************
Loss

******************** Train ********************
Loss: 0.059893202036619186, Accuracy: 0.9575757384300232 % 	 time=0.14s
******************** Train ********************
Loss: 0.0038470954168587923, Accuracy: 0.9636363387107849 % 	 time=0.13s
******************** Train ********************
Loss: 0.025020595639944077, Accuracy: 0.9696969985961914 % 	 time=0.14s
******************** Train ********************
Loss: 0.029019635170698166, Accuracy: 0.9757575988769531 % 	 time=0.14s
******************** Train ********************
Loss: 0.009000107645988464, Accuracy: 0.939393937587738 % 	 time=0.15s
******************** Train ********************
Loss: 0.043232373893260956, Accuracy: 0.9575757384300232 % 	 time=0.14s
******************** Train ********************
Loss: 0.10947819799184799, Accuracy: 0.903030276298523 % 	 time=0.14s
******************** Train ********************
Loss: 0.052809350192546844, Accuracy: 0.9696969985961914 % 	 time=0.13s
******************** Train ***************

In [213]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(trn_test_loader)*batch_size)))

tensor([[9.9770e-01],
        [7.7920e-06],
        [4.5067e-06],
        [5.0004e-09],
        [1.4452e-05]])
tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([[1.0406e-02],
        [1.9704e-04],
        [1.5216e-05],
        [1.2154e-04],
        [9.5568e-01]])
tensor([0, 0, 0, 0, 1], dtype=torch.uint8)
tensor([[5.3070e-04],
        [4.1634e-03],
        [5.9836e-04],
        [4.7112e-03],
        [9.8084e-01]])
tensor([0, 0, 0, 0, 1], dtype=torch.uint8)
tensor([[5.5454e-06],
        [9.9575e-01],
        [9.9786e-01],
        [8.1745e-04],
        [4.7511e-04]])
tensor([0, 1, 1, 0, 0], dtype=torch.uint8)
tensor([[2.3659e-08],
        [3.9443e-04],
        [9.7846e-01],
        [1.5387e-05],
        [4.4059e-08]])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([[4.6777e-03],
        [9.8442e-01],
        [1.0633e-03],
        [9.7996e-01],
        [7.4645e-04]])
tensor([0, 1, 0, 1, 0], dtype=torch.uint8)
tensor([[8.3881e-03],
        [4.0452e-04],
        [9.7773e-01],
        [2

In [214]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]
#trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
pd.DataFrame(trn_y_pd, predict_.iloc[:,0])

,0
0,
9.976984e-01,1.0
7.792013e-06,0.0
4.506733e-06,0.0
5.000369e-09,0.0
1.445213e-05,0.0
...,...
5.661125e-04,0.0
6.239417e-04,0.0
4.808797e-05,0.0


In [215]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(trn_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print('AUC: ', auc(fpr, trp))

[[ 47   0]
 [  0 121]]
Accuracy :  1.0
Sensitivity :  1.0
Specificity :  1.0
AUC:  1.0


# Final Test

In [216]:
model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())
        #print(val_pred)
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        #predicted = torch.max(val_pred.data, 1)[1]
        predicted = (val_pred >= 0.5).flatten().type(torch.ByteTensor)
        print(predicted)
        print(val_y)
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

tensor([1, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 1.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 0, 1, 0, 0], dtype=torch.uint8)
tensor([0., 0., 1., 0., 0.])
tensor([1, 0, 1, 0, 0], dtype=torch.uint8)
tensor([1., 0., 1., 0., 0.])
tensor([0, 1, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 1., 0.])
tensor([0, 0, 0, 1, 1], dtype=torch.uint8)
tensor([0., 0., 0., 0., 1.])
tensor([1, 0, 0, 0, 0], dtype=torch.uint8)
tensor([0., 1., 0., 0., 0.])
tensor([0, 1, 1, 0, 1], dtype=torch.uint8)
tensor([0., 0., 1., 0., 1.])
tensor([1, 1], dtype=torch.uint8)
tensor([1., 1.])
******************** Test ********************
Loss: 0.15460722148418427, Accuracy: 0.7333333492279053 %
**********************************************
Val accuracy:0.733


In [217]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
8.274139e-01,0.0
8.693683e-01,1.0
4.560153e-09,0.0
1.307053e-06,0.0
2.900085e-10,1.0
2.794144e-04,0.0
1.154467e-04,1.0
9.570977e-01,0.0
2.324721e-01,0.0


In [218]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

label = pd.DataFrame(val_y_pd)

test_p = pd.concat([predict_.iloc[:,0], label], axis = 1)

#print(test_p)

test_p.columns = ['predict','label']


test_p.loc[test_p['predict'] >= 0.5, 'predicted_prob'] = 1
test_p.loc[test_p['predict'] < 0.5, 'predicted_prob'] = 0


#print(test_p)

cnf = confusion_matrix(test_p['label'], test_p['predicted_prob'], labels = [1,0])

print(cnf)

total1 = sum(sum(cnf))

accuracy1=(cnf[0,0]+cnf[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cnf[0,0]/(cnf[0,0]+cnf[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cnf[1,1]/(cnf[1,0]+cnf[1,1])
print('Specificity : ', specificity1)

fpr, trp, _ = roc_curve(test_p['label'], test_p['predicted_prob'])

print(type(fpr))
#print(fpr)
print('AUC: ', auc(fpr, trp))

[[10  4]
 [ 5 23]]
Accuracy :  0.7857142857142857
Sensitivity :  0.7142857142857143
Specificity :  0.8214285714285714
<class 'numpy.ndarray'>
AUC:  0.7678571428571429


# Model Save

# require to check~~

In [74]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_200_100_100_model_1_100_250__1.pth")     

In [169]:
torch.save({
#'epoch': EPOCHS,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': trn_loss
    }, "./platin_model_save/platin_model_500_model_2_200_300.pth")    

In [526]:
trn_X_pd

array([[1.59662362e-05, 0.00000000e+00, 1.02076500e-04, ...,
        0.00000000e+00, 1.28752303e-02, 1.23934200e-04],
       [7.74798159e-07, 3.41406163e-05, 1.16552895e-06, ...,
        0.00000000e+00, 4.21414921e-05, 2.42437585e-05],
       [6.97953402e-06, 1.15329500e-04, 2.95293198e-06, ...,
        0.00000000e+00, 5.33686870e-05, 2.74985439e-05],
       ...,
       [3.73017687e-05, 6.57464300e-04, 3.46030501e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.34393991e-05, 5.16415500e-04, 2.42411746e-05, ...,
        0.00000000e+00, 9.87277592e-06, 1.45977700e-04],
       [6.58817034e-08, 9.67667916e-07, 2.47764632e-08, ...,
        0.00000000e+00, 1.33158250e-03, 8.23836230e-03]])

In [215]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [216]:
batch_size = 5

In [217]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)


In [218]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [219]:
tmp = next(iter(trn_loader))

In [220]:
tmp

{'X': tensor([[6.5677e-05, 8.3347e-02, 3.8531e-05,  ..., 1.9207e-03, 3.3437e-04,
          4.4619e-04],
         [1.9899e-05, 2.3116e-03, 0.0000e+00,  ..., 0.0000e+00, 9.3073e-05,
          1.6355e-04],
         [4.5190e-07, 1.4934e-05, 3.8238e-07,  ..., 0.0000e+00, 5.9395e-04,
          3.6635e-03],
         [3.8192e-04, 3.8086e-01, 4.2333e-04,  ..., 0.0000e+00, 2.8409e-04,
          1.2932e-03],
         [1.9966e-05, 7.4559e-06, 1.2918e-05,  ..., 0.0000e+00, 6.3502e-04,
          1.8207e-02]], dtype=torch.float64),
 'y': tensor([0., 1., 0., 0., 1.], dtype=torch.float64)}

In [221]:
num_batches = len(trn_loader)

## Build Model

In [222]:
use_cuda = torch.cuda.is_available()

In [223]:
class MLPRegressor(nn.Module):
    
    def __init__(self):
        super(MLPRegressor, self).__init__()
        h1 = nn.Linear((len(train.columns)-3), 32)
        h2 = nn.Linear(32, 16)
        h3 = nn.Linear(16, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        if use_cuda:
            self.hidden = self.hidden.cuda()
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1)

In [224]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
         
        self.layer1 = nn.Sequential(
            torch.nn.Linear((len(train.columns)-3), 256, bias=True),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU()
        )
 
        self.layer2 = nn.Sequential(
            torch.nn.Linear(256, 128, bias=True),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU()
        )
    
        self.layer3 = nn.Sequential(
            torch.nn.Linear(128, 64, bias=True),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU()
        )
 
        self.layer4 = nn.Sequential(
            torch.nn.Linear(64, 1, bias=True),
            
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.layer1(x)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer2(x_out)
        x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer3(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer4(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [332]:
class DNN_seq_(nn.Module):
    def __init__(self):
        super(DNN_seq_, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
        )
        self.classifier2 = nn.Sequential(

            torch.nn.Linear(100, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(100,1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [259]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq(nn.Module):
    def __init__(self):
        super(DNN_seq, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(150, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(250, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            torch.nn.Linear(250, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [245]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight) 

In [246]:
model = DNN_seq()

In [247]:
model.apply(weights_init)

DNN_seq(
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=150, bias=True)
    (1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=150, out_features=250, bias=True)
    (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (classifier3): Sequential(
    (0): Linear(in_features=250, out_features=250, bias=True)
    (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=250, out_features=250, bias=True)
    (5): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=250, ou

## Train model

In [248]:
criterion = nn.BCELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [249]:
num_epochs = 20
num_batches = len(trn_loader) # requier modification

In [250]:
num_batches

36

In [251]:
trn_X_pd.shape[1]

400

# Train_2

In [252]:
model = DNN_seq()

In [253]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)#,lr=0.001, betas=(0.9,0.999))
    error = nn.BCELoss()
    num_epochs = 20
    
    #checkpoint = torch.load("./platin_model_save/First_platin_model.pth")                                                                                  
    #model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
    #num_epochs = checkpoint['epoch']                                                                                                           
    #loss = checkpoint['loss']        
    
    
    model.train()
    for epoch in range(num_epochs):
        trn_loss_summary = 0.0
        correct = 0   # Accuracy

        for batch_idx, trn in enumerate(trn_loader):
            trn_X, trn_y = trn['X'], trn['y']
            if use_cuda:
                trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
            trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
            optimizer.zero_grad()
            trn_pred = model(trn_X)
            #trn_pred = trn_pred.type()
            #type_as(torch.cuda.IntTensor())
            trn_loss = criterion(trn_pred.squeeze(), trn_y)
            trn_loss.backward()
            optimizer.step()
        
            trn_loss_summary += trn_loss
        
            predicted = torch.max(trn_pred.data, 1)[1] 
            correct += (predicted == trn_y).sum()
            #print(correct)
            if batch_idx % 2 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(trn_X), len(trn_loader.dataset), 100.*batch_idx / len(trn_loader), trn_loss.data, float(correct*100) / float(batch_size*(batch_idx+1))))
    torch.save({
    #'epoch': EPOCHS,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': trn_loss
        }, "./platin_model_save/platin_model_2.pth")    

In [254]:
#model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/180 (0%)]	Loss: 0.776929	 Accuracy:80.000%
Epoch : 0 [10/180 (6%)]	Loss: 0.801617	 Accuracy:80.000%
Epoch : 0 [20/180 (11%)]	Loss: 0.696659	 Accuracy:76.000%
Epoch : 0 [30/180 (17%)]	Loss: 0.769450	 Accuracy:74.286%
Epoch : 0 [40/180 (22%)]	Loss: 0.563943	 Accuracy:75.556%
Epoch : 0 [50/180 (28%)]	Loss: 0.615975	 Accuracy:76.364%
Epoch : 0 [60/180 (33%)]	Loss: 0.621697	 Accuracy:72.308%
Epoch : 0 [70/180 (39%)]	Loss: 0.754056	 Accuracy:68.000%
Epoch : 0 [80/180 (44%)]	Loss: 0.327390	 Accuracy:68.235%
Epoch : 0 [90/180 (50%)]	Loss: 0.497569	 Accuracy:69.474%
Epoch : 0 [100/180 (56%)]	Loss: 0.407298	 Accuracy:72.381%
Epoch : 0 [110/180 (61%)]	Loss: 0.496243	 Accuracy:72.174%
Epoch : 0 [120/180 (67%)]	Loss: 0.654800	 Accuracy:71.200%
Epoch : 0 [130/180 (72%)]	Loss: 0.469441	 Accuracy:70.370%
Epoch : 0 [140/180 (78%)]	Loss: 0.593599	 Accuracy:71.034%
Epoch : 0 [150/180 (83%)]	Loss: 0.512662	 Accuracy:71.613%
Epoch : 0 [160/180 (89%)]	Loss: 0.658217	 Accuracy:70.303%
Epoch : 0 

Epoch : 8 [30/180 (17%)]	Loss: 0.306748	 Accuracy:68.571%
Epoch : 8 [40/180 (22%)]	Loss: 0.185420	 Accuracy:71.111%
Epoch : 8 [50/180 (28%)]	Loss: 0.471007	 Accuracy:72.727%
Epoch : 8 [60/180 (33%)]	Loss: 0.256661	 Accuracy:72.308%
Epoch : 8 [70/180 (39%)]	Loss: 0.274515	 Accuracy:74.667%
Epoch : 8 [80/180 (44%)]	Loss: 0.561807	 Accuracy:74.118%
Epoch : 8 [90/180 (50%)]	Loss: 0.323444	 Accuracy:71.579%
Epoch : 8 [100/180 (56%)]	Loss: 0.683152	 Accuracy:71.429%
Epoch : 8 [110/180 (61%)]	Loss: 0.569845	 Accuracy:71.304%
Epoch : 8 [120/180 (67%)]	Loss: 0.381507	 Accuracy:72.000%
Epoch : 8 [130/180 (72%)]	Loss: 0.238484	 Accuracy:72.593%
Epoch : 8 [140/180 (78%)]	Loss: 0.253200	 Accuracy:72.414%
Epoch : 8 [150/180 (83%)]	Loss: 0.248830	 Accuracy:71.613%
Epoch : 8 [160/180 (89%)]	Loss: 0.293989	 Accuracy:71.515%
Epoch : 8 [170/180 (94%)]	Loss: 0.649500	 Accuracy:71.429%
Epoch : 9 [0/180 (0%)]	Loss: 0.588557	 Accuracy:40.000%
Epoch : 9 [10/180 (6%)]	Loss: 0.303254	 Accuracy:66.667%
Epoch : 9

Epoch : 16 [40/180 (22%)]	Loss: 2.041943	 Accuracy:57.778%
Epoch : 16 [50/180 (28%)]	Loss: 0.523941	 Accuracy:65.455%
Epoch : 16 [60/180 (33%)]	Loss: 0.372615	 Accuracy:63.077%
Epoch : 16 [70/180 (39%)]	Loss: 0.314646	 Accuracy:64.000%
Epoch : 16 [80/180 (44%)]	Loss: 0.347928	 Accuracy:68.235%
Epoch : 16 [90/180 (50%)]	Loss: 0.128908	 Accuracy:70.526%
Epoch : 16 [100/180 (56%)]	Loss: 0.631151	 Accuracy:69.524%
Epoch : 16 [110/180 (61%)]	Loss: 0.184766	 Accuracy:68.696%
Epoch : 16 [120/180 (67%)]	Loss: 0.259879	 Accuracy:68.800%
Epoch : 16 [130/180 (72%)]	Loss: 0.405108	 Accuracy:69.630%
Epoch : 16 [140/180 (78%)]	Loss: 0.653986	 Accuracy:71.034%
Epoch : 16 [150/180 (83%)]	Loss: 0.166828	 Accuracy:71.613%
Epoch : 16 [160/180 (89%)]	Loss: 0.140924	 Accuracy:71.515%
Epoch : 16 [170/180 (94%)]	Loss: 0.225211	 Accuracy:71.429%
Epoch : 17 [0/180 (0%)]	Loss: 0.425675	 Accuracy:60.000%
Epoch : 17 [10/180 (6%)]	Loss: 0.384020	 Accuracy:66.667%
Epoch : 17 [20/180 (11%)]	Loss: 0.150301	 Accuracy:

In [255]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Train', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_loader)*batch_size)))
print('*'*46)
print("Train accuracy:{:.3f}".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.4637147784233093, Accuracy: 0.7166666388511658 %
**********************************************
Train accuracy:0.717


In [256]:

model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())

        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 1.1907775402069092, Accuracy: 0.6285714507102966 %
**********************************************
Val accuracy:0.629


In [243]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.396110,0.0
0.776959,0.0
0.542691,0.0
0.377997,0.0
0.365426,0.0
0.380689,1.0
0.145689,1.0
0.157259,0.0
0.366938,0.0


In [260]:
model = DNN_seq()

In [261]:
checkpoint = torch.load("./platin_model_save/First_platin_model.pth")                                                                                  
model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
#num_epochs = checkpoint['epoch']                                                                                                           
loss = checkpoint['loss']  

In [262]:
use_cuda = torch.cuda.is_available()

In [263]:

model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())

        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += torch.sum(predicted == val_y).item()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

RuntimeError: size mismatch, m1: [5 x 400], m2: [500 x 150] at /opt/conda/conda-bld/pytorch_1570710822989/work/aten/src/TH/generic/THTensorMath.cpp:197

In [28]:
val_pred.data

tensor([[0.1228],
        [0.0049]])

In [24]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.000213,0.0
0.587282,0.0
0.206532,0.0
0.767732,0.0
0.000056,0.0
0.826447,0.0
0.965310,1.0
0.000252,0.0
0.451954,0.0


# Visualization

In [41]:
plt.figure(figsize=(16,9))
x_range = range(len(trn_loss_list))
plt.plot(x_range, trn_loss_list, label="trn")
plt.plot(x_range, val_loss_list, label="val")
plt.legend()
plt.xlabel("training steps")
plt.ylabel("loss")

NameError: name 'trn_loss_list' is not defined

<Figure size 1152x648 with 0 Axes>

# Transfer Learning

결론적으로 이미 잘 훈련된 모델이 있고, 특히 해당 모델과 유사한 문제를 해결시 transfer learining을 사용합니다.
실질적 조언
새로 훈련할 데이터가 적지만 original 데이터와 유사할 경우

데이터의 양이 적어 fine-tune (전체 모델에 대해서 backpropagation을 진행하는 것) 은 over-fitting의 위험이 있기에 하지 않습니다.
새로 학습할 데이터는 original 데이터와 유사하기 때문에 이 경우 최종 linear classfier 레이어만 학습을 합니다.
새로 훈련할 데이터가 매우 많으며 original 데이터와 유사할 경우

새로 학습할 데이터의 양이 많다는 것은 over-fitting의 위험이 낮다는 뜻이므로, 전체 레이어에 대해서 fine-tune을 합니다.
새로 훈련할 데이터가 적으며 original 데이터와 다른 경우

데이터의 양이 적기 때문에 최종 단계의 linear classifier 레이어를 학습하는 것이 좋을 것입니다. 반면서 데이터가 서로 다르기 때문에 거의 마지막부분 (the top of the network)만 학습하는 것은 좋지 않습니다. 서로 상충이 되는데.. 이 경우에는 네트워크 초기 부분 어딘가 activation 이후에 특정 레이어를 학습시키는게 좋습니다.
새로 훈련할 데이터가 많지만 original 데이터와와 다른 경우

데이터가 많기 때문에 아예 새로운 ConvNet을 만들수도 있지만, 실적적으로 transfer learning이 더 효율이 좋습니다. 전체 네트워크에 대해서 fine-tune을 해도 됩니다.

In [75]:
model.classifier3

Sequential(
  (0): Linear(in_features=32, out_features=16, bias=True)
  (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
)

In [149]:
model_2 = DNN_seq()

In [150]:
for param in model.parameters():
    param.requires_grad = True

model.classifier = model_2.classifier    
model.classifier2 = model_2.classifier2


In [151]:
#model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.746858	 Accuracy:50.000%
Epoch : 0 [32/171 (18%)]	Loss: 0.662493	 Accuracy:60.417%
Epoch : 0 [64/171 (36%)]	Loss: 0.704260	 Accuracy:60.000%
Epoch : 0 [96/171 (55%)]	Loss: 0.674394	 Accuracy:66.071%
Epoch : 0 [128/171 (73%)]	Loss: 0.722818	 Accuracy:67.361%
Epoch : 0 [110/171 (91%)]	Loss: 0.599336	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.575366	 Accuracy:87.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.667310	 Accuracy:70.833%
Epoch : 1 [64/171 (36%)]	Loss: 0.673750	 Accuracy:68.750%
Epoch : 1 [96/171 (55%)]	Loss: 0.751240	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.740712	 Accuracy:67.361%
Epoch : 1 [110/171 (91%)]	Loss: 0.710288	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.641428	 Accuracy:62.500%
Epoch : 2 [32/171 (18%)]	Loss: 0.618767	 Accuracy:68.750%
Epoch : 2 [64/171 (36%)]	Loss: 0.619703	 Accuracy:65.000%
Epoch : 2 [96/171 (55%)]	Loss: 0.579154	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.667362	 Accuracy:66.667%
Epoch : 2 [110/

In [152]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5934396982192993, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


# Voting

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
# VotingClassifier
from sklearn.ensemble import VotingClassifier

# ensemble 할 model 정의
models = [
    ('ada', AdaBoostClassifier()),
    ('bc', BaggingClassifier()),
    ('etc',ExtraTreesClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('rfc', RandomForestClassifier()),
    ('knn', KNeighborsClassifier()),
    ('svc', SVC(probability=True)),
    ('xgb', XGBClassifier()),
    ('lgbm', LGBMClassifier()),
    ('dtc', DecisionTreeClassifier()),
    ('lr', LogisticRegressionCV()),
    ('ridge', RidgeClassifier()),
]

# hard vote
hard_vote  = VotingClassifier(models, voting='hard')
hard_vote_cv = cross_validate(hard_vote, x_train, y_train, cv=k_fold)
hard_vote.fit(x_train, y_train)

# soft vote
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote_cv = cross_validate(soft_vote, x_train, y_train, cv=k_fold)
soft_vote.fit(x_train, y_train)


In [76]:
model_1 = DNN_seq_()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_1 = model_1.cuda()
    
fit(model_1, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.935273	 Accuracy:68.750%
Epoch : 0 [32/171 (18%)]	Loss: 0.791408	 Accuracy:70.833%
Epoch : 0 [64/171 (36%)]	Loss: 0.752347	 Accuracy:68.750%
Epoch : 0 [96/171 (55%)]	Loss: 0.723812	 Accuracy:63.393%
Epoch : 0 [128/171 (73%)]	Loss: 0.811310	 Accuracy:65.972%
Epoch : 0 [110/171 (91%)]	Loss: 0.819974	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.823073	 Accuracy:62.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.663486	 Accuracy:64.583%
Epoch : 1 [64/171 (36%)]	Loss: 0.554099	 Accuracy:66.250%
Epoch : 1 [96/171 (55%)]	Loss: 0.667068	 Accuracy:66.071%
Epoch : 1 [128/171 (73%)]	Loss: 0.624170	 Accuracy:65.278%
Epoch : 1 [110/171 (91%)]	Loss: 0.637554	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.651228	 Accuracy:75.000%
Epoch : 2 [32/171 (18%)]	Loss: 0.534523	 Accuracy:81.250%
Epoch : 2 [64/171 (36%)]	Loss: 0.607727	 Accuracy:71.250%
Epoch : 2 [96/171 (55%)]	Loss: 0.625725	 Accuracy:69.643%
Epoch : 2 [128/171 (73%)]	Loss: 0.518599	 Accuracy:68.750%
Epoch : 2 [110/

In [77]:
model_2 = DNN_seq_()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_2 = model_2.cuda()
    
fit(model_2, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.745150	 Accuracy:56.250%
Epoch : 0 [32/171 (18%)]	Loss: 0.593750	 Accuracy:70.833%
Epoch : 0 [64/171 (36%)]	Loss: 0.609681	 Accuracy:72.500%
Epoch : 0 [96/171 (55%)]	Loss: 0.811276	 Accuracy:70.536%
Epoch : 0 [128/171 (73%)]	Loss: 0.909858	 Accuracy:68.750%
Epoch : 0 [110/171 (91%)]	Loss: 0.555978	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.671668	 Accuracy:62.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.751559	 Accuracy:60.417%
Epoch : 1 [64/171 (36%)]	Loss: 0.629498	 Accuracy:66.250%
Epoch : 1 [96/171 (55%)]	Loss: 0.660346	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.670158	 Accuracy:68.056%
Epoch : 1 [110/171 (91%)]	Loss: 0.560989	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.624824	 Accuracy:68.750%
Epoch : 2 [32/171 (18%)]	Loss: 0.593675	 Accuracy:70.833%
Epoch : 2 [64/171 (36%)]	Loss: 0.510769	 Accuracy:67.500%
Epoch : 2 [96/171 (55%)]	Loss: 0.542457	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.449139	 Accuracy:68.056%
Epoch : 2 [110/

In [78]:
criterion = nn.BCELoss()

In [79]:
model_1.eval()
predict_1 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_1(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_1.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5453603863716125, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [80]:
criterion = nn.BCELoss()

In [81]:
model_2.eval()
predict_2 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_2(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_2.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.586574375629425, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [69]:
predict_1

[array([0.39712372], dtype=float32),
 array([0.20998392], dtype=float32),
 array([0.32124352], dtype=float32),
 array([0.34580824], dtype=float32),
 array([0.43426463], dtype=float32),
 array([0.43412134], dtype=float32),
 array([0.39131775], dtype=float32),
 array([0.38226905], dtype=float32),
 array([0.42771617], dtype=float32),
 array([0.2733672], dtype=float32),
 array([0.42271107], dtype=float32),
 array([0.4628345], dtype=float32),
 array([0.4519838], dtype=float32),
 array([0.3947924], dtype=float32),
 array([0.39574006], dtype=float32),
 array([0.4404809], dtype=float32),
 array([0.3974393], dtype=float32),
 array([0.48102662], dtype=float32),
 array([0.42520103], dtype=float32),
 array([0.43733662], dtype=float32),
 array([0.3043311], dtype=float32),
 array([0.4567577], dtype=float32),
 array([0.36690694], dtype=float32),
 array([0.38817948], dtype=float32),
 array([0.5215086], dtype=float32),
 array([0.4337628], dtype=float32),
 array([0.40996504], dtype=float32),
 array([0.6

In [70]:
predict_2

[array([0.40936574], dtype=float32),
 array([0.30449998], dtype=float32),
 array([0.37039232], dtype=float32),
 array([0.37293124], dtype=float32),
 array([0.45267642], dtype=float32),
 array([0.44899222], dtype=float32),
 array([0.41215253], dtype=float32),
 array([0.40887272], dtype=float32),
 array([0.4442977], dtype=float32),
 array([0.39285985], dtype=float32),
 array([0.4360027], dtype=float32),
 array([0.48552424], dtype=float32),
 array([0.46931365], dtype=float32),
 array([0.40514487], dtype=float32),
 array([0.41182923], dtype=float32),
 array([0.42791894], dtype=float32),
 array([0.405836], dtype=float32),
 array([0.4981986], dtype=float32),
 array([0.43440622], dtype=float32),
 array([0.44471112], dtype=float32),
 array([0.37168515], dtype=float32),
 array([0.47441038], dtype=float32),
 array([0.3651678], dtype=float32),
 array([0.40067512], dtype=float32),
 array([0.5199345], dtype=float32),
 array([0.4233134], dtype=float32),
 array([0.4351335], dtype=float32),
 array([0.

In [71]:
#predict_3 = predict_1 + predict_2

In [72]:
len(predict_3)

NameError: name 'predict_3' is not defined

In [56]:
type(predict_1)

list

In [92]:
import operator

predict_3 = map(operator.add, predict_1, predict_2)

In [93]:
print(predict_3)

In [94]:
predict = []

for i in predict_3:
    print (i/2)
    predict.append(i/2)
    print (predict)



[0.43645966]
[array([0.43645966], dtype=float32)]
[0.34106642]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32)]
[0.4075713]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32)]
[0.40122908]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32)]
[0.44226646]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32)]
[0.44394046]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32)]
[0.41578606]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0

In [97]:
print(predict)
type(predict)

[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32), array([0.41578606], dtype=float32), array([0.41548812], dtype=float32), array([0.4436235], dtype=float32), array([0.357646], dtype=float32), array([0.43614274], dtype=float32), array([0.4559007], dtype=float32), array([0.44684273], dtype=float32), array([0.42056525], dtype=float32), array([0.42621326], dtype=float32), array([0.42557824], dtype=float32), array([0.4176554], dtype=float32), array([0.45774207], dtype=float32), array([0.43780085], dtype=float32), array([0.4308337], dtype=float32), array([0.4050982], dtype=float32), array([0.4504068], dtype=float32), array([0.39789623], dtype=float32), array([0.41764146], dtype=float32), array([0.4750628], dtype=float32), array([0.4315881], dtype=float32), array([0.4379266], dtype=float32), array([0.4739846], dtype=float32), arra

list

In [104]:
val_y_pd_ = val_y_pd.to_numpy()
val_y_pd_ = torch.from_numpy(val_y_pd_)
val_y_pd_ = val_y_pd_.type_as(torch.FloatTensor())

In [ ]:
val_loss = criterion(predict, val_y)
val_loss_summary += val_loss
predicted = torch.max(val_pred.data, 1)[1] 
correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

In [96]:
val_y_pd

0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    1.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    1.0
27    0.0
28    0.0
29    1.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    1.0
37    1.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
Name: label, dtype: float64

In [93]:
#num_ftrs = model.forward.in_features
 
for name, param in model.named_parameters():
    print (name, param.data)

layer1.0.weight tensor([[-0.0565, -0.1205,  0.0296,  ...,  0.2230, -0.0025,  0.0114],
        [-0.0879, -0.0202, -0.0235,  ...,  0.0587,  0.0370, -0.0111],
        [-0.0381, -0.0511, -0.0141,  ..., -0.0142,  0.0329,  0.0550],
        ...,
        [ 0.0349, -0.0490, -0.0410,  ..., -0.0243, -0.0495,  0.0048],
        [ 0.0487,  0.1016, -0.0085,  ..., -0.1036,  0.0225, -0.0505],
        [-0.0129,  0.1087,  0.0454,  ..., -0.1197,  0.0375,  0.0101]])
layer1.0.bias tensor([-0.0296, -0.0339,  0.0066, -0.0166, -0.0347, -0.0489,  0.0233,  0.0351,
         0.0348,  0.0116,  0.0404, -0.0197, -0.0154,  0.0126,  0.0236,  0.0361,
        -0.0178, -0.0001, -0.0028, -0.0274, -0.0495, -0.0317,  0.0020,  0.0040,
         0.0019,  0.0077,  0.0134, -0.0460,  0.0494,  0.0400, -0.0072, -0.0119,
         0.0044, -0.0115,  0.0237, -0.0051, -0.0136, -0.0101,  0.0195,  0.0419,
        -0.0333, -0.0286,  0.0231, -0.0157,  0.0131,  0.0034, -0.0038,  0.0113,
        -0.0342,  0.0022, -0.0403,  0.0045,  0.0368, -0.

In [95]:
model.layer1 = 

NameError: name 'Sequential' is not defined

In [79]:
nn.Linear(num_ftrs)

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [32]:
list(model.children())[0:3]

AttributeError: 'list' object has no attribute 'parameter'

## Train model

In [77]:
model = MLPRegressor()

NameError: name 'X_features' is not defined

In [78]:
criterion = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'model' is not defined

In [44]:
batch_size = 32

In [45]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

val = data_utils.TensorDataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

NameError: name 'val_X' is not defined

In [13]:
trn_X_pd = df2_mod.iloc[:,2:]

In [14]:
trn_X_pd

ID,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,ENSG00000109321,ENSG00000112499,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
TCGA-61-1910,0.970585,0.003561,0.044476,0.000000,0.000000,0.000000,0.043732,2.308592,1.014494,0.000000,...,0.000000,0.920597,0.000000,1.604301,0.299376,0.000000,0.894232,1.612154,0.761720,0.000000
TCGA-24-0982,0.395276,0.005010,0.090384,0.007062,0.000000,0.006438,0.082034,0.242661,0.691196,0.004156,...,24.797152,0.259035,4.625172,0.188089,4.211878,0.270091,0.034947,0.311310,0.357217,0.000000
TCGA-36-1580,1.403594,0.008641,0.133845,0.001970,0.002559,0.041762,0.074384,0.984290,2.329158,0.001391,...,4.017149,0.289083,0.893372,0.314861,4.817979,0.000000,0.000000,0.707253,15.996040,0.000000
TCGA-25-1321,0.308600,0.003129,0.004478,0.162223,0.000788,0.000000,0.101272,0.240456,1.740308,0.004819,...,2.576079,0.166843,0.000000,0.121147,1.356422,0.521892,0.157563,2.033770,1.495528,0.000000
TCGA-30-1892,0.206074,0.133216,0.119614,0.056072,0.000000,0.055381,0.280458,0.592872,0.914734,0.004400,...,1.058604,0.114269,2.825069,0.000000,3.716018,0.000000,0.000000,0.294278,1.890974,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-04-1347,0.319288,0.000491,0.033696,0.005704,0.000988,0.011701,0.060742,0.663423,0.094262,0.000671,...,0.000000,0.000000,0.000000,3.038608,0.567030,0.000000,0.000000,0.071847,4.616723,0.000000
TCGA-24-0968,0.237050,0.009708,0.185222,0.000000,0.000000,0.000000,0.307004,0.213121,1.291373,0.003796,...,15.221570,0.525782,4.332951,2.720169,31.845646,0.205584,0.000000,0.710875,6.117782,0.000000
TCGA-30-1891,0.772034,0.000481,0.110194,0.000000,0.000646,0.200678,0.000000,0.098616,2.959271,0.077290,...,47.331214,0.000000,2.631492,0.000000,65.568652,0.000000,0.000000,0.000000,80.395413,0.000000
TCGA-09-0366,0.195435,0.000085,1.631170,0.026487,0.001376,0.006339,0.303848,0.703636,5.424154,0.008418,...,0.225023,0.534375,0.100086,1.481522,3.712525,0.000000,0.058985,2.477118,0.602935,0.001573


In [15]:
trn_y_pd = df2_mod.iloc[:,1]

In [16]:
trn_y_pd

TCGA-61-1910    0.007666
TCGA-24-0982    0.007190
TCGA-36-1580    0.027081
TCGA-25-1321    0.002315
TCGA-30-1892    0.057091
                  ...   
TCGA-04-1347    0.000000
TCGA-24-0968    0.013682
TCGA-30-1891    0.034186
TCGA-09-0366    0.032361
TCGA-13-0884    0.018597
Name: ENSG00000048545, Length: 214, dtype: float64

In [17]:
trn_X = torch.from_numpy(trn_X_pd.astype(float).values)
trn_y = torch.from_numpy(trn_y_pd.astype(float).values)

In [18]:
batch_size=64

In [19]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

In [20]:
tmp = next(iter(trn_loader))

In [21]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)

In [22]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)